In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
mylist = []

for chunk in  pd.read_csv('zomato.csv', chunksize=2000):
    mylist.append(chunk)

zomato_real = pd.concat(mylist, axis= 0)
del mylist

In [ ]:
zomato_real = pd.DataFrame()
for chunk in pd.read_csv('zomato.csv', chunksize=1000):
    zomato_real = pd.concat([zomato_real, chunk], ignore_index=True)

In [ ]:
zomato_real.head()

In [ ]:
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1)

In [ ]:
zomato.duplicated().sum()

In [ ]:
zomato.drop_duplicates(inplace=True)

In [ ]:
zomato.head()

In [ ]:
zomato.isnull().sum()

In [ ]:
zomato.dropna(inplace=True)

In [ ]:
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})

In [ ]:
zomato.head()

In [ ]:
zomato['cost'] = zomato['cost'].astype(str)

In [ ]:
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.'))

In [ ]:
zomato['cost'] = zomato['cost'].astype(float)

In [ ]:
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)

In [ ]:
zomato.head()

In [ ]:
remove_slash = lambda x: x.replace('/5','') 
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

In [ ]:
zomato.name = zomato.name.apply(lambda x:x.title())

In [ ]:
zomato.head()

In [ ]:
zomato.online_order = zomato.online_order.replace('Yes',True)
zomato.online_order = zomato.online_order.replace('No',False)

In [ ]:
zomato.info()

In [ ]:
list(zomato.name.unique())

In [ ]:
zomato.shape

In [ ]:
zomato.name.value_counts()

In [ ]:
zomato.columns

In [ ]:
restaurants = list(zomato['name'].unique())
zomato['mean_ratings'] = 0


In [ ]:
for i in range(len(restaurants)):
    zomato['mean_ratings'][zomato['name']==restaurants[i]] = zomato['rate'][zomato['name']==restaurants[i]].mean()

In [ ]:
zomato.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(1,5))
zomato[['mean_ratings']] = scaler.fit_transform(zomato[['mean_ratings']]).round(2)

In [ ]:
zomato.head()

In [ ]:
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

In [ ]:
import string
punc_to_remove = string.punctuation

In [ ]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','',punc_to_remove))

In [ ]:
zomato['reviews_list'] = zomato['reviews_list'].apply(remove_punctuation)

In [ ]:
zomato.head()

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

In [ ]:
len(stopwords)

In [ ]:
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords])
    

In [ ]:
zomato['reviews_list'] = zomato['reviews_list'].apply(remove_stopwords)

In [ ]:
zomato.head()

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
zomato['reviews_list'] = zomato['reviews_list'].apply(remove_urls)

In [ ]:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

In [ ]:
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [ ]:
df_percent = zomato.sample(frac=1)

In [ ]:
df_percent

In [ ]:
df_percent.set_index('name', inplace=True)

In [ ]:
indices = pd.Series(df_percent.index)

In [ ]:
indices

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,1), stop_words='english', min_df=0)
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])
# cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
tfidf_matrix

In [ ]:
np.multiply(tfidf_matrix, np.transpose(tfidf_matrix))

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
tfidf_matrix.shape

In [ ]:

def recommend(name, cosine_similarities = cosine_similarities):
    
    recommend_restaurant = []
    
    
    idx = indices[indices == name].index[0]
    
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
   
    top30_indexes = list(score_series.iloc[0:31].index)
    
   
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
 
    df_new = pd.DataFrame(columns=['cuisines', 'mean_ratings', 'cost','location'])
    
   
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','mean_ratings', 'cost','location']][df_percent.index == each].sample()))
    
    
    df_new = df_new.drop_duplicates(subset=['cuisines','mean_ratings', 'cost','location'], keep=False)
    df_new = df_new.sort_values(by='mean_ratings', ascending=False)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new

In [ ]:
recommend('Shawarma Inc')

In [ ]:
list(indices)